<a href="https://colab.research.google.com/github/itberrios/CV_tracking/blob/main/kitti_tracker/0_kitti_gps_imu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **KITTI GPS/IMU**


In this tutorial we will obtain high precision GPS/IMU data from the [KITTI](http://www.cvlibs.net/datasets/kitti/raw_data.php) dataset. The experiment used a OXTS RT3003 inertial and GPS navigation system to obtain this data. For more information a readme for the KITTI data can be found [here](https://github.com/yanii/kitti-pcl/blob/master/KITTI_README.TXT), and a paper that details the data collection and coordinate systems can be found [here](http://www.cvlibs.net/publications/Geiger2013IJRR.pdf)

This tutorial will provide the framework for future tutorials that will involve tracking objects in the real world as opposed to pixel space.

The first thing to do is to get the data

In [2]:
!wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_10_03_drive_0047/2011_10_03_drive_0047_sync.zip

--2022-09-09 12:18:15--  https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_10_03_drive_0047/2011_10_03_drive_0047_sync.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.169.133
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.169.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3103291675 (2.9G) [application/zip]
Saving to: ‘2011_10_03_drive_0047_sync.zip’

2011_10_03_drive_00 100%[===================>]   2.89G  33.2MB/s    in 96s     

2022-09-09 12:19:50 (31.0 MB/s) - ‘2011_10_03_drive_0047_sync.zip’ saved [3103291675/3103291675]



In [3]:
!wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_10_03_calib.zip

--2022-09-09 12:19:51--  https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_10_03_calib.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.47.75
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.47.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4075 (4.0K) [application/zip]
Saving to: ‘2011_10_03_calib.zip’

2011_10_03_calib.zi 100%[===================>]   3.98K  --.-KB/s    in 0s      

2022-09-09 12:19:51 (162 MB/s) - ‘2011_10_03_calib.zip’ saved [4075/4075]



In [4]:
!jar xf 2011_10_03_drive_0047_sync.zip
!jar xf 2011_10_03_calib.zip

The GPS/IMU data is located in the oxts folder, we can explore the outputs and even plot the path of the vehicle during the video.

In [7]:
import os
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (20, 10)

Let's view the headers for the GPS/IMU data

In [56]:
DATA_PATH = '2011_10_03/2011_10_03_drive_0047_sync'
oxts_header = pd.read_csv(
    glob(os.path.join(DATA_PATH, r'**/*dataformat.txt'))[0], 
    header=None,
    sep='\t')

oxts_header

,0
0,lat: latitude of the oxts-unit (deg)
1,lon: longitude of the oxts-unit (deg)
2,alt: altitude of the oxts-unit (m)
3,"roll: roll angle (rad), 0 = level, positiv..."
4,"pitch: pitch angle (rad), 0 = level, positiv..."
5,"yaw: heading (rad), 0 = east, positiv..."
6,vn: velocity towards north (m/s)
7,ve: velocity towards east (m/s)
8,"vf: forward velocity, i.e. parallel to eart..."
9,"vl: leftward velocity, i.e. parallel to ear..."


We see that the we get two main frames of reference a global frame and alocal frame that have different represtnations of the same measurement. For this tutorial we will only consider position and linear velocity.

- Global 
  - Position in Latitude, Longitude, and Altitude
  - Velocity in North, East, and Up

- Local
  - Velocity in Left, Forward, and Up

Now that we know which datapoints we need, let's read in all the oxt files! We will need both time stamps and the oxts data

In [73]:
LAT = 0
LON = 1
VEL_N = 6
VEL_E = 7
VEL_F = 8
VEL_L = 9

In [96]:
oxts_timestamps = pd.read_csv(
    glob(os.path.join(DATA_PATH, r'oxts**/*timestamps.txt'))[0], 
    header=None
    )

oxts_timestamps

,0
0,2011-10-03 14:57:27.036448963
1,2011-10-03 14:57:27.146615260
2,2011-10-03 14:57:27.246339073
3,2011-10-03 14:57:27.346351297
4,2011-10-03 14:57:27.456352889
...,...
832,2011-10-03 14:58:53.256599714
833,2011-10-03 14:58:53.356553061
834,2011-10-03 14:58:53.466548861
835,2011-10-03 14:58:53.566558102


In [ ]:
# remove date from timestamps
# oxts_timestamps = oxts_timestamps[0].apply(lambda x: x.split(' ')[1])

In [47]:
oxts_data_paths = sorted(glob(os.path.join(DATA_PATH, r'oxts/data**/*.txt')))

In [163]:
def get_oxts(oxt_path):
    ''' Obtains the oxt info from a single oxt path '''
    with open(oxt_path) as f:
        oxts = f.readlines()
        
    oxts = oxts[0].strip().split(' ')
    oxts = np.array(oxts).astype(float)
    
    return oxts

In [164]:
oxts_data = list(map(get_oxts, oxts_data_paths))

## Now let's display where the vehicle has traveled on a real map

We will use [folium](https://python-visualization.github.io/folium/modules.html) for this

In [ ]:
!pip install folium

In [186]:
import folium

drive_map = folium.Map(
    location = (oxts_data[0][LAT], oxts_data[0][LON]), # starting location
    zoom_start=15
)

In [187]:
# now add markers for each lat lon position of the vehicle

folium.Marker(
    location=(oxts_data[0][LAT], oxts_data[0][LON]),
    popup='Start',
    icon=folium.Icon(color='green')
).add_to(drive_map)

folium.Marker(
    location=(oxts_data[-1][LAT], oxts_data[-1][LON]),
    popup='Stop',
    icon=folium.Icon(color='red')
).add_to(drive_map);


In [188]:
# add every position update to the map
points = []
for oxts in oxts_data:
    points.append((oxts[LAT], oxts[LON]))

folium.PolyLine(points, color='blue', weight=8).add_to(drive_map);

In [189]:
drive_map

Now we have an interactive map that documents the drive.